# Motor de Investigación

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [26]:
# %pip install langchain langchain_community langchain_openai duckduckgo-search

In [2]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
load_dotenv()

True

## Búsqueda en la web

Pasos a realizar:
1. Crear una herramienta de búsqueda en la web.
2. Crear una herramienta de web scraping.
3. Crear los prompts.
4. Implementar la arquitectura.

### Paso 1: Crear una herramienta de búsqueda en la web

In [3]:
# Herramienta de búsqueda en la web

from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from typing import List
 
def web_search(web_query: str, num_results: int) -> List[str]:
    return [r["link"] for r in DuckDuckGoSearchAPIWrapper().results(web_query, num_results)]

In [4]:
# Hagamos una prueba del buscador
 
result = web_search(web_query="Cuántos balones de oro tiene Messi?", num_results=5)
print(result)

['https://www.dazn.com/es-ES/news/fútbol/cuantos-balones-oro-leo-messi/lqeidd5kuq1111h3n3zxixkyu', 'https://www.marca.com/futbol/balon-oro/2024/10/28/cuantos-balones-oro-leo-messi-cristiano-ronaldo.html', 'https://www.ole.com.ar/usa/futbol-internacional/balones-oro-lionel-messi-cristiano-ronaldo-maximos-ganadores-historia-balon-oro-2024_0_GQ7ekTVxfC.html', 'https://posiciondegol.com/cuantos-balones-de-oro-tiene-messi', 'https://as.com/futbol/internacional/messi-vs-cristiano-cuantos-balones-de-oro-ha-ganado-cada-uno-n-3/']


Nota 
--- 
LangChain también ofrece otros wrappers para motores de búsqueda web, como **TavilySearchResults** y **GoogleSearchAPIWrapper**. Pero ambos requieren una clave API, por practicidad decidimos usar **DuckDuckGoSearchAPIWrapper**.

### Paso 2: Crear una herramienta de web scraping

In [5]:
# Herramienta de web scraping

import requests
from bs4 import BeautifulSoup
 
def web_scrape(url: str) -> str:
    try:
        response = requests.get(url)
 
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator=" ", strip=True)
 
            return page_text
        else:
            return f"No se pudo recuperar la página web: Código {response.status_code}"
    except Exception as e:
        print(e)
        return f"No se pudo recuperar la página web: {e}"

In [6]:
# Vamos a probar la herramienta de web scraping

result = web_scrape('https://www.sport.es/es/noticias/balon-oro/balones-oro-messi-total-12264298')
print(result)

¿Cuántos Balones de Oro tiene Messi en total? Temas Asamblea Real Madrid Florentino Balón de Oro Verstappen campeón Frenkie de Jong Barcelona - Tenerife Leganés - Real Madrid Alineación Real Madrid Suegro Verstappen Dinero Verstappen Clasificación F1 Clasificación LaLiga Jorge Martín Sainz Leclerc Messi Barcelona España Nations League La Revuelta Polémica Comunicado Hormiguero Copa Davis hoy Despedida Nadal Policia Nacional Rodri Hernández Dónde ver Copa Davis La Revuelta Clasificación Champions F Clasificación LaLiga Clasificación Premier League Cruces Champions Dinero Barcelona Champions Clasificación Champions Dónde ver Premier Padel Mundial de Clubes Clasificación Youth League Clasificación Euroliga Premier Padel Ranking Últimas noticias Bota de Oro Formato Champions Dinero Champions Fichajes Barça Dónde ver LaLiga Clasificación F1 Próxima carrera F1 Red Bull Batalla Clasificación MotoGP Carnet pensionista Suscripción sport ÚLTIMA HORA Barça Barça Clásico Fichajes Última hora Femen

### Paso 3: Crear los prompts

In [7]:
from langchain.prompts import PromptTemplate

instrucciones_asistente_seleccion = """
Eres experto en asignar una pregunta de investigación al asistente de investigación correcto.
Hay varios asistentes de investigación disponibles, cada uno especializado en un área de experiencia.
Cada asistente está identificado por un tipo específico y tiene instrucciones específicas para llevar a cabo la investigación.

Cómo seleccionar el asistente correcto: debes seleccionar el asistente relevante dependiendo del tema de la pregunta, que debe coincidir con el área de experiencia del asistente.

------
Aquí tienes algunos ejemplos de cómo devolver la información correcta del asistente, según la pregunta realizada.

Ejemplos:
Pregunta: "¿Debería invertir en títulos de cobertura?"
Respuesta: 
{{
    "tipo_asistente": "Asistente analista financiero",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial experto en análisis financiero. Tu objetivo principal es elaborar informes financieros completos, perspicaces, imparciales y organizados metódicamente basados en los datos y tendencias proporcionados.",
    "pregunta_usuario": {pregunta_usuario}
}}
Pregunta: "¿Cuáles son los sitios más interesantes en Río de Janeiro?"
Respuesta: 
{{
    "tipo_asistente": "Asistente de guía turístico",
    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

Pregunta: "¿Es Messi un buen jugador de fútbol?"
Respuesta: 
{{
    "tipo_asistente": "Asistente experto en deportes",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial especializado en deportes. Tu propósito principal es redactar informes deportivos atractivos, perspicaces, imparciales y bien estructurados sobre personalidades deportivas o eventos deportivos, incluyendo detalles factuales, estadísticas y análisis.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

------
Ahora que has entendido todo lo anterior, selecciona el asistente de investigación correcto para la siguiente pregunta.
Pregunta: {pregunta_usuario}
Respuesta:

""" 

plantilla_asistente_seleccion = PromptTemplate.from_template(
    template=instrucciones_asistente_seleccion
)


In [8]:
instrucciones_busqueda_web = """
{instrucciones_asistente}

Escribe {num_consultas_busqueda} consultas de búsqueda web para recopilar la mayor cantidad de información posible 
sobre la siguiente pregunta: {pregunta_usuario}. Tu objetivo es escribir un informe basado en la información que encuentres.
Debes responder con una lista de consultas como consulta1, consulta2, consulta3 en el siguiente formato: 
[
    {{"consulta_busqueda": "consulta1", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta2", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta3", "pregunta_usuario": "{pregunta_usuario}" }}
]
"""

plantilla_busqueda_web = PromptTemplate.from_template(
    template=instrucciones_busqueda_web
)


In [9]:
instrucciones_resumen = """
Lee el siguiente texto:
Texto: {texto_resultado_busqueda}

-----------

Usando el texto anterior, responde brevemente la siguiente pregunta.
Pregunta: {consulta_busqueda}

Si no puedes responder la pregunta anterior usando el texto proporcionado, simplemente resume el texto.

Incluye toda la información factual, números, estadísticas, etc., si están disponibles.
"""

plantilla_resumen = PromptTemplate.from_template(
    template=instrucciones_resumen
)


In [10]:
# Prompt de informe de investigación adaptado de https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py

instrucciones_informe_investigacion = """
Eres un asistente de investigación de pensamiento crítico impulsado por IA. Tu único propósito es escribir informes bien redactados, aclamados críticamente, objetivos y estructurados sobre el texto proporcionado.

Información: 
--------
{resumen_investigacion}
--------

Usando la información anterior, responde la siguiente pregunta o tema: "{pregunta_usuario}" en un informe detallado -- \
El informe debe centrarse en la respuesta a la pregunta, estar bien estructurado, ser informativo, \
profundo, con hechos y cifras si están disponibles, y tener un mínimo de 1,200 palabras.

Debes esforzarte por escribir el informe lo más extenso posible utilizando toda la información relevante y necesaria proporcionada.
Debes escribir el informe con sintaxis de markdown.
DEBES determinar tu propia opinión concreta y válida basada en la información proporcionada. NO infieras conclusiones generales y sin sentido.
Escribe todas las URLs de las fuentes utilizadas al final del informe y asegúrate de no agregar fuentes duplicadas, solo una referencia por cada una.
Debes escribir el informe en formato APA."""
 
plantilla_informe_investigacion = PromptTemplate.from_template(
    template=instrucciones_informe_investigacion
)


### Paso 4: Implementar la arquitectura

In [11]:
# Creemos la instancia del LLM
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    temperature=0.6,
)

In [12]:
# Constantes

NUMERO_CONSULTAS_BUSQUEDA = 2
NUMERO_RESULTADOS_POR_CONSULTA = 3
MAXIMO_CARACTERES_TEXTO_RESULTADO = 10000

In [13]:
# El único input que necesitamos es la pregunta del usuario

pregunta = "Qué se puede hacer en la Isla de Margarita?"

El primer paso de la arquitectura es ejecutar el prompt del LLM para determinar el asistente de investigación correcto basadas en la pregunta de investigación del usuario

In [14]:
prompt_asistente_seleccion = plantilla_asistente_seleccion.format(pregunta_usuario=pregunta)

instrucciones_asistente = llm.invoke(prompt_asistente_seleccion)

# Veamos el resultado
print(instrucciones_asistente.content)

{
    "tipo_asistente": "Asistente de guía turístico",
    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",
    "pregunta_usuario": "Qué se puede hacer en la Isla de Margarita?"
}


In [15]:
# Convertimos el resultado en un objeto
import json

dict_instrucciones_asistente = json.loads(instrucciones_asistente.content)
dict_instrucciones_asistente

{'tipo_asistente': 'Asistente de guía turístico',
 'instrucciones_asistente': 'Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.',
 'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'}

El siguiente paso en la arquitectura es ejecutar el prompt de búsqueda web para recopilar información relevante sobre la pregunta de investigación del usuario

In [16]:
prompt_busqueda_web = plantilla_busqueda_web.format(instrucciones_asistente=dict_instrucciones_asistente['instrucciones_asistente'],
                                                    num_consultas_busqueda=NUMERO_CONSULTAS_BUSQUEDA,
                                                    pregunta_usuario=dict_instrucciones_asistente['pregunta_usuario'])
consultas_busqueda_web = llm.invoke(prompt_busqueda_web)
lista_consultas_busqueda_web = json.loads(consultas_busqueda_web.content)
lista_consultas_busqueda_web


[{'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'guía de viaje Isla de Margarita experiencias recomendadas',
  'pregunta_usuario': 'Qué se puede hacer en la Isla de Margarita?'}]

In [17]:
# Vamos a realizar la búsqueda web
resultados_busquedas = [{'urls_resultado': web_search(web_query=wq['consulta_busqueda'], 
                                     num_results=NUMERO_RESULTADOS_POR_CONSULTA), 
                           'consulta_busqueda': wq['consulta_busqueda']} 
                           for wq in lista_consultas_busqueda_web]


In [18]:
resultados_busquedas

[{'urls_resultado': ['https://www.lugaresturisticos.org/atractivos-turisticos-de-la-isla-de-margarita/',
   'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
   'https://www.cosasdeviajes.com/que-ver-en-isla-margarita/'],
  'consulta_busqueda': 'actividades turísticas Isla de Margarita'},
 {'urls_resultado': ['https://www.lugaresturisticos.org/atractivos-turisticos-de-la-isla-de-margarita/',
   'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
   'https://www.ldhoteles.com/guia-de-viaje-para-la-isla-de-margarita'],
  'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita'},
 {'urls_resultado': ['https://www.ldhoteles.com/guia-de-viaje-para-la-isla-de-margarita',
   'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
   'https://hospedamargarita.com/los-mejores-consejos-para-viajar-a-la-isla-de-margarita-en-venezuela/'],
  'consulta_busqueda': 'guía de viaje Isla d

Cada consulta de búsqueda web tiene una lista de URLs de resultados. Ahora, necesitamos extraer el texto de cada URL para resumirlo y responder a la pregunta de investigación del usuario.

In [19]:
# Vamos a "aplanar" la lista de resultados de búsqueda

resultados_busquedas_aplanado = []
for qr in resultados_busquedas:
    resultados_busquedas_aplanado.extend([{'consulta_busqueda': qr['consulta_busqueda'], 
                                    'urls_resultado': r
                                    } for r in qr['urls_resultado']])

In [20]:
resultados_busquedas_aplanado

[{'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.lugaresturisticos.org/atractivos-turisticos-de-la-isla-de-margarita/'},
 {'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/'},
 {'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'urls_resultado': 'https://www.cosasdeviajes.com/que-ver-en-isla-margarita/'},
 {'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita',
  'urls_resultado': 'https://www.lugaresturisticos.org/atractivos-turisticos-de-la-isla-de-margarita/'},
 {'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita',
  'urls_resultado': 'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/'},
 {'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita',
  'urls_resultado': 'https://www.ldhoteles.com/guia-de-viaje-pa

In [21]:
# Ahora usaremos la función de web scraping para extraer el texto de cada URL

textos_resultados = [ {'texto_resultado_busqueda': web_scrape(url=re['urls_resultado'])[:MAXIMO_CARACTERES_TEXTO_RESULTADO],
                     'urls_resultado': re['urls_resultado'],
                     'consulta_busqueda': re['consulta_busqueda']}
                   for re in resultados_busquedas_aplanado]

In [22]:
textos_resultados

[{'texto_resultado_busqueda': '▷ 10 Mejores Atractivos Turísticos de la Isla de Margarita - ¿Qué visitar? (2024) Facebook Instagram Twitter Vimeo VKontakte Youtube América del Sur Argentina Bolivia Brasil Chile Colombia Ecuador Perú Paraguay Uruguay Venezuela América Central Costa Rica El Salvador Guatemala Honduras Nicaragua Panamá América del Norte Canadá Estados Unidos México Europa Alemania Bélgica España Francia Grecia Holanda Italia Portugal Suiza Asia Bangladesh Camboya China Corea del Sur Filipinas India Indonesia Japón Malasia Nepal Singapur Tailandia Taiwán Vietnam Africa Marruecos Samoa Seychelles Sudáfrica Zambia Buscar viernes, noviembre 22, 2024 Contacto Partners Registrarse ¡Bienvenido! Ingresa en tu cuenta tu nombre de usuario tu contraseña ¿Olvidaste tu contraseña? consigue ayuda Recuperación de contraseña Recupera tu contraseña tu correo electrónico Se te ha enviado una contraseña por correo electrónico. Lugares Turísticos América del Sur Argentina Bolivia Brasil Chil

El paso siguiente es resumir cada página. Este resumen mantendrá la consulta y las urls porque las necesitaremos para el informe de investigación.

In [23]:
lista_resumen_texto = []


# Lo hacemos secuencialmente, pero se puede hacer en paralelo
for rt in textos_resultados: 
    prompt_resumen = plantilla_resumen.format(
        texto_resultado_busqueda=rt['texto_resultado_busqueda'], 
        consulta_busqueda=rt['consulta_busqueda'])
    
    respuesta_llm = llm.invoke(prompt_resumen)
    resumen_texto = respuesta_llm.content
 
    lista_resumen_texto.append({'resumen_texto': resumen_texto,
                         'urls_resultado': rt['urls_resultado'],
                         'consulta_busqueda': rt['consulta_busqueda']})

In [24]:
lista_resumen_texto

[{'resumen_texto': 'La Isla de Margarita ofrece diversas actividades turísticas que destacan por su belleza natural y riqueza cultural. Algunas de las principales actividades incluyen:\n\n1. **Relajarse en playas**: Playa El Agua es famosa por sus arenas blancas y aguas cristalinas, ideal para deportes acuáticos y paseos. Playa Parguito es conocida por sus olas, siendo un lugar popular para surfistas.\n\n2. **Explorar parques nacionales**: El Parque Nacional Laguna de La Restinga permite paseos en bote a través de manglares, mientras que el Parque Nacional Cerro El Copey ofrece senderismo y vistas panorámicas.\n\n3. **Visitar sitios históricos**: El Castillo de San Carlos de Borromeo en Pampatar y el Castillo de Santa Rosa en La Asunción son importantes patrimonios que brindan información sobre la historia colonial de la isla.\n\n4. **Disfrutar de la gastronomía local**: Probar platos típicos como las empanadas de cazón en playas y pueblos.\n\n5. **Contemplar atardeceres**: Juan Griego

Finalmente, el último paso es escribir el informe de investigación utilizando los resúmenes de cada página web.

In [25]:
# Vamos a convertir en string el objeto lista_resumen_texto
resumenes_string = json.dumps(lista_resumen_texto)

# Creamos el prompt para el informe de investigación
prompt_reporte_investigacion = plantilla_informe_investigacion.format(
    resumen_investigacion=resumenes_string,
    pregunta_usuario=pregunta
)

# Generamos el informe de investigación

respuesta_llm = llm.invoke(prompt_reporte_investigacion)

reporte_investigacion = respuesta_llm.content


# Veamos el resultado
print(reporte_investigacion)

# Qué se puede hacer en la Isla de Margarita

La Isla de Margarita, situada en el estado de Nueva Esparta, Venezuela, se destaca como un destino turístico de gran atractivo en el Caribe. Conocida por su clima cálido, playas de arena blanca y una rica cultura, la isla ofrece una amplia variedad de actividades que abarcan desde la relajación en sus hermosas playas hasta la exploración de su patrimonio histórico y cultural. Este informe detalla las principales actividades que se pueden disfrutar en la Isla de Margarita, organizadas en diferentes categorías para facilitar la comprensión.

## 1. Actividades en la Playa

### 1.1 Playa El Agua

Playa El Agua es una de las más famosas y visitadas de la isla, con una extensión de aproximadamente 4 kilómetros de arena dorada y aguas cristalinas. Este lugar no solo es ideal para relajarse bajo el sol, sino que también es perfecto para practicar deportes acuáticos como el windsurf, el kitesurf y el buceo. La playa está rodeada de restaurantes y ba